# DataBase Loaders

1. SQLDatabaseLoader

Definition:
A loader that connects to SQL databases (Postgres, MySQL, SQLite, etc.) and extracts table rows into documents for use with LLMs.

Usage:
Use this when you want to bring raw SQL database content into LangChain for retrieval or fine-tuning.

Parameters:

sql_database: SQLAlchemy SQLDatabase object

query: SQL query (string) to fetch data

Code Example:

## First Create SQLite DB

In [18]:
import sqlite3


conn = sqlite3.connect("11_sqlite_db.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    income REAL,
    spending_score INTEGER
);
""")

# Insert dummy data
dummy_customers = [
    ("Ali", 25, 30000, 45),
    ("Sara", 32, 60000, 80),
    ("Ahmed", 40, 120000, 20),
    ("Ayesha", 29, 45000, 60),
    ("Bilal", 50, 90000, 30)
]
cursor.executemany("INSERT INTO customers (name, age, income, spending_score) VALUES (?, ?, ?, ?);", dummy_customers)

conn.commit()
conn.close()

### Now Load With SQLDatabaseLoader

In [19]:
from langchain_community.document_loaders import SQLDatabaseLoader
from langchain_community.utilities import SQLDatabase

# Step 1: Connect to your existing DB 
db = SQLDatabase.from_uri("sqlite:///11_sqlite_db.db")

# Step 2: Load data with SQLDatabaseLoader
loader = SQLDatabaseLoader(db=db,  query="SELECT * FROM customers;")

docs = loader.load()

# Step 3: Print results 
for doc in docs:
    print(doc.page_content)

id: 1
name: Ali
age: 25
income: 30000.0
spending_score: 45
id: 2
name: Sara
age: 32
income: 60000.0
spending_score: 80
id: 3
name: Ahmed
age: 40
income: 120000.0
spending_score: 20
id: 4
name: Ayesha
age: 29
income: 45000.0
spending_score: 60
id: 5
name: Bilal
age: 50
income: 90000.0
spending_score: 30
id: 6
name: Ali
age: 25
income: 30000.0
spending_score: 45
id: 7
name: Sara
age: 32
income: 60000.0
spending_score: 80
id: 8
name: Ahmed
age: 40
income: 120000.0
spending_score: 20
id: 9
name: Ayesha
age: 29
income: 45000.0
spending_score: 60
id: 10
name: Bilal
age: 50
income: 90000.0
spending_score: 30


# 2. MongoDBLoader

Definition:
A loader that connects to MongoDB collections and converts documents into LangChain Document objects.

Usage:
Use this when working with NoSQL (document-oriented) databases like MongoDB.

Parameters:

connection_string: MongoDB URI

db_name: Database name

collection_name: Collection to load documents from

filter: (optional) MongoDB filter query

Code Example:

# First Create Mongo DB

In [20]:
from pymongo import MongoClient

#  Step 1: Connect to MongoDB 
client = MongoClient("mongodb://localhost:27017/")

#  Step 2: Create database and collection 
db = client["testdb"]
collection = db["customers"]

#  Step 3: Insert dummy data 
dummy_customers = [
    {"name": "Ali", "age": 25, "income": 30000, "status": "active"},
    {"name": "Sara", "age": 32, "income": 60000, "status": "active"},
    {"name": "Ahmed", "age": 40, "income": 120000, "status": "inactive"},
    {"name": "Ayesha", "age": 29, "income": 45000, "status": "active"},
    {"name": "Bilal", "age": 50, "income": 90000, "status": "inactive"}
]

collection.insert_many(dummy_customers)

print("MongoDB test database created with dummy customers.")


MongoDB test database created with dummy customers.


# Load Data Using MongodbLoader

In [ ]:
from langchain_community.document_loaders.mongodb import MongodbLoader

loader = MongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="testdb",
    collection_name="customers",
    field_names=["name", "age", "income", "status"] 
)

docs = loader.load()

for doc in docs:
    print("Content:", doc.page_content)

Content: Ali 25 30000 active
Content: Sara 32 60000 active
Content: Ahmed 40 120000 inactive
Content: Ayesha 29 45000 active
Content: Bilal 50 90000 inactive
Content: Ali 25 30000 active
Content: Sara 32 60000 active
Content: Ahmed 40 120000 inactive
Content: Ayesha 29 45000 active
Content: Bilal 50 90000 inactive
